# Issuer Agent

In [ ]:
import requests
import json
import qrcode
import uuid
import base64
import re
import jwt
import time
# https://pypi.org/project/termcolor/
from termcolor import colored,cprint

In [ ]:
# Set the URL for the API endpoint
# Need to specify the host as host.docker.internal as the Prism Agents and the Prism Playground are running in two isolated docker environments
base_url = "http://host.docker.internal:8080"

headers = {'Content-Type': 'application/json', 'accept': 'application/json'}

## Check Agent Records
1. Connections
2. Credentials
3. Proof Presentations

In [ ]:
print(colored('Agent State:\n', attrs=['bold']))
path = '/prism-agent/connections'

# Make the request
response = requests.get(base_url + path, headers=headers)

print(colored('Connection Records:\n', attrs=['bold']))
# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    
path = '/prism-agent/issue-credentials/records'

print(colored('Credential Records:\n', attrs=['bold']))
# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)
    
path = '/prism-agent/present-proof/presentations'

print(colored('Proof Presentation Records:\n', attrs=['bold']))
# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

## Make a Connection with Holder Agent

### Create a connection record containing the invitation
```bash
curl -X 'POST' \
	'http://localhost:8080/prism-agent/connections' \
	-H 'Content-Type: application/json' \
	-d '{
		"label": "Connect with Alice"
		}' | jq
```

In [ ]:
path = '/prism-agent/connections'
data = {"label": "Connect with Alice"}
# Make the request
response = requests.post(base_url + path, headers=headers, json=data)

# Check the response status code
if response.status_code == 201:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-201 status code")

### Send invitation to Holder

In [ ]:
invitation = json.loads(response.content.decode())
invitationUrl = invitation['invitation']['invitationUrl']
print(invitationUrl)
invitationUrlbase64 = re.sub("^.*_oob=", "", invitationUrl)
# print(invitationUrlbase64)
connection_id = invitation['connectionId']
%store invitationUrlbase64


### Retrieving the list of connections
```bash
curl -X 'GET' 'http://localhost:8080/prism-agent/connections' | jq
```

In [ ]:
STATE = None

path = '/prism-agent//connections/' + connection_id

# print('Current state for ConnectionId {} is {}'.format(connection_id,STATE))
print(colored("Current state for ConnectionId {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))
while STATE != 'ConnectionResponseSent':
    # Make the request
    response = requests.get(base_url + path, headers=headers)

    # Check the response status code
    if response.status_code == 200:
        # Print the response content
        loaded_json = json.loads(response.content.decode())
        # print(json.dumps(loaded_json, indent=2))
        STATE = loaded_json['state']
        print(colored("ConnectionId {0} is not in active state yet".format(connection_id), "yellow", attrs=["bold"]))
        print(colored("State: {0}".format(STATE), "blue", attrs=["bold"]))

        try:
            their_did = loaded_json['theirDid']
        except (KeyError) as e:
            print(colored("Holder didn't accept invitation yet. Head over to Holder notebook and accept connection invitation".format(connection_id,STATE), "red", attrs=["bold"]))

    else:
        # Print an error message
        print("Error: The API returned a non-200 status code")
        

    time.sleep(1)
    
# print('ConnectionId: {0} is now active. Continue with notebook'.format(connection_id))
print(colored("ConnectionId: {0} is now active. Continue with notebook".format(connection_id), "green", attrs=["bold"]))


### Generate QR code from invitation

In [ ]:
# Link for connection invitation
input_data = invitationUrl
# Creating an instance of qrcode
qr = qrcode.QRCode(
        version=1,
        box_size=10,
        border=5)
qr.add_data(input_data)
qr.make(fit=True)
img = qr.make_image(fill='black', back_color='white')
img.save('issuer_agent_invite_QRcode.png')

from IPython.display import Image
Image(width=400, filename='./issuer_agent_invite_QRcode.png')

## Issue Credential to Holder

### Initiate a new issue credential flow

Replace `{SUBJECT_ID}` with the DID of the holder displayed at startup in the his Prism Agent console logs
```bash
curl -X 'POST' \
  'http://localhost:8080/prism-agent/issue-credentials/credential-offers' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
      "schemaId": "schema:1234",
      "subjectId": "{SUBJECT_ID}",
      "validityPeriod": 3600,
      "automaticIssuance": false,
      "awaitConfirmation": false,
      "claims": {
        "firstname": "Alice",
        "lastname": "Wonderland",
        "birthdate": "01/01/2000"
      }
	}' | jq
```

In [ ]:
# subject_id = 'did:peer:2.Ez6LSsyLCEveJGeF9jWGkBCG6wvfzXcNgu5A8PAiKp3NzvPdb.Vz6MkuZxR1VDeXiJ9uL9Uuh7Rh58DGpYw3abdtacLv8KdL2sP.SeyJ0IjoiZG0iLCJzIjoiaHR0cDovL2hvc3QuZG9ja2VyLmludGVybmFsOjgwOTAvZGlkY29tbS8iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19'
path = '/prism-agent/issue-credentials/credential-offers'
data = {
      "schemaId": "schema:1234",
      "subjectId": their_did,
      "validityPeriod": 3600,
      "automaticIssuance": False,
      "awaitConfirmation": False,
      "claims": {
        "firstname": "Alice",
        "lastname": "Wonderland",
        "birthdate": "01/01/2000"
      }
    }

# Make the request
response = requests.post(base_url + path, headers=headers, json=data)

# Check the response status code
if response.status_code == 201:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    STATE = loaded_json['protocolState']
else:
    # Print an error message
    print("Error: The API returned a non-201 status code")
    print(response.content)

In [ ]:
credential_record_id = loaded_json['recordId']
%store credential_record_id

In [ ]:
path = '/prism-agent/issue-credentials/records'

print('Current state for Credential {} is {}'.format(connection_id,STATE))
print(colored("Current state for Credential Issuance {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))
while STATE != 'RequestReceived':
    # Make the request
    response = requests.get(base_url + path, headers=headers)

    # Check the response status code
    if response.status_code == 200:
        # Print the response content
        loaded_json = json.loads(response.content.decode())
        # print(json.dumps(loaded_json, indent=2))
        STATE = loaded_json['items'][0]['protocolState']
        print(colored("ConnectionId {0}".format(connection_id), "yellow", attrs=["bold"]))
        print(colored("Credential Issuance State: {0}".format(STATE), "blue", attrs=["bold"]))

        # try:
        #     their_did = loaded_json['theirDid']
        # except (KeyError) as e:
        #     print(colored("Holder didn't accept invitation yet".format(connection_id,STATE), "red", attrs=["bold"]))

    else:
        # Print an error message
        print("Error: The API returned a non-200 status code")
        

    time.sleep(1)
    
# print('ConnectionId: {0} is now active. Continue with notebook'.format(connection_id))
print(colored("CredentialId: {0} is now in correct state. Continue to Issue Credential".format(connection_id), "green", attrs=["bold"]))


### - Issuing the credential

> Replace `{RECORD_ID}` with the UUID of the record from the previous list
```bash
curl -X 'POST' 'http://localhost:8080/prism-agent/issue-credentials/records/{RECORD_ID}/issue-credential' | jq
```

In [ ]:
path = '/prism-agent/issue-credentials/records/' + credential_record_id + '/issue-credential'
# Make the request
response = requests.post(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

In [ ]:
path = '/prism-agent/issue-credentials/records'

print('Current state for Credential {} is {}'.format(connection_id,STATE))
print(colored("Current state for Credential Issuance {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))
while STATE != 'CredentialSent':
    # Make the request
    response = requests.get(base_url + path, headers=headers)

    # Check the response status code
    if response.status_code == 200:
        # Print the response content
        loaded_json = json.loads(response.content.decode())
        # print(json.dumps(loaded_json, indent=2))
        STATE = loaded_json['items'][0]['protocolState']
        print(colored("ConnectionId {0}".format(connection_id), "yellow", attrs=["bold"]))
        print(colored("Credential Issuance State: {0}".format(STATE), "blue", attrs=["bold"]))

        # try:
        #     their_did = loaded_json['theirDid']
        # except (KeyError) as e:
        #     print(colored("Holder didn't accept invitation yet".format(connection_id,STATE), "red", attrs=["bold"]))

    else:
        # Print an error message
        print("Error: The API returned a non-200 status code")
        

    time.sleep(1)
    
# print('ConnectionId: {0} is now active. Continue with notebook'.format(connection_id))
print(colored("CredentialId: {0} Issued!.".format(connection_id), "green", attrs=["bold"]))


## DIDs

### Create a new unpublised DID stored in Prism Agent
```bash
curl --location --request POST 'http://localhost:8080/prism-agent/did-registrar/dids' \
  --header 'Content-Type: application/json' \
  --header 'Accept: application/json' \
  --data-raw '{
    "documentTemplate": {
      "publicKeys": [
        {
          "id": "auth0",
          "purpose": "authentication"
        }
      ],
      "services": []
    }
  }'
```

In [ ]:
path = '/prism-agent/did-registrar/dids'
data = {
    "documentTemplate": {
      "publicKeys": [
        {
          "id": "auth0",
          "purpose": "authentication"
        }
      ],
      "services": []
    }
  }

# Make the request
response = requests.post(base_url + path, headers=headers, json=data)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

### Lists all the DIDs in Prism Agent
```bash
curl --location --request GET 'http://localhost:8080/prism-agent/did-registrar/dids' \
  --header 'Accept: application/json'
```



In [ ]:
path = '/prism-agent/did-registrar/dids'

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    DID_REF = loaded_json[0]['longFormDid']
    DID_REF_SHORT = loaded_json[0]['did']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

### Publishes the DID in Prism Agent to the blockchain
Replace `DID_REF` by the DID on Prism Agent that should be published
```bash
curl --location --request POST 'http://localhost:8080/prism-agent/did-registrar/dids/{DID_REF}/publications' \
--header 'Accept: application/json'
```

In [ ]:
path = '/prism-agent/did-registrar/dids/' + DID_REF + '/publications'

# Make the request
response = requests.post(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 202:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-202 status code")
    print(response.content)

### Resolves the DID document of Prism DID
Replace `DID_REF` by the DID on Prism Agent that should be resolved
```bash
curl --location --request GET 'http://localhost:8080/prism-agent/dids/{DID_REF}' \
--header 'Accept: application/json'
```

In [ ]:
path = '/prism-agent/dids/' + DID_REF

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)